In [1]:
import json
import os
from dotenv import load_dotenv
from openai import OpenAI
from pyairtable import Api

# Load the environment variables from the .env file
load_dotenv()
# Assuming you have an Airtable API key and base ID
api_key = os.getenv('AIRTABLE_API_KEY')
base_id = os.getenv('AIRTABLE_BASE')


client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')
)

# Create an instance of the Airtable class
api = Api(api_key)

In [88]:
table = api.table(base_id, 'people')
descrip = api.table(base_id, 'descs')

# Get all records from the table
records = table.all()

In [41]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [84]:
import base64

def summarize_object_with_chatgpt(obj):
    # Convert the object to a JSON string
    obj_str = f"""
    These are person X's responses to the questions:

    1. What’s the best way to get to know you as a person?
    (Give an unstructured response, anything works!)
    {obj.get("as_a_person")}

    2. What traits do you envy/value in those around you?
    (try to be as honest as possible)
    {obj.get("traits")}

    3. What do you value in friendships? What are the best ways they add to your life?
    {obj.get("value_in_friendships")}

    4. Anything else?
    (list out literally anything else, we'll use AI to match you to the best person (music taste, sports, literally anything)
    if you want, include a pitch of yourself here and we'll send it to them!)
    {obj.get("other")}

    5. What are you looking for in a partner?
    (study buddy, reading friend, or a long term husband???)
    {obj.get("looking for")}

    6. If you were arrested for a crime, what would it be???
    {obj.get("crime")}
    """
    base64_image = encode_image("screenshot.png")

    try:
        # Generate a summary using the OpenAI ChatGPT API in turbo mode
        response = client.chat.completions.create(
            model="gpt-4-vision-preview",  # Adjust the model if necessary
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url":f"data:image/jpeg;base64,{base64_image}"
                        },
                    }
                ]},
                {"role": "user", "content": f"Generate a compelling summary of person X as a whole. Make logical generalizations. Make it one paragraph. \n\n afterwards give a paragraph analysis of the insta profile above & the vibe. then provide a list of physical attributes: {obj_str}"}
            ],
            temperature=0.7,
            max_tokens=300,  # Adjust based on how long you expect the summary to be
        )

        response2 = client.chat.completions.create(
            model="gpt-4-vision-preview",  # Adjust the model if necessary
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url":f"data:image/jpeg;base64,{base64_image}"
                        },
                    }
                ]},
                {"role": "user", "content": f"Generate a holistic description for who the ideal partner for person X would be. Include specifics. DO NOT DESCRIBE PERSON X AT ALL. Make it one paragraph. use generalizations from the image above.: {obj_str}"}
            ],
            temperature=0.7,
            max_tokens=300,  # Adjust based on how long you expect the summary to be
        )

        # Extract and return the summary text
        # Ensure the response structure is navigated correctly based on the API's response format
        return (response.choices[0].message.content.strip(), response2.choices[0].message.content.strip())
    except Exception as e:
        return f"An error occurred: {str(e)}"

In [91]:
import time
from selenium import webdriver

def download_insta(user):
    # Create a new instance of the Chrome driver
    options = webdriver.FirefoxOptions()
    driver = webdriver.Firefox()

    # Navigate to the website
    driver.get(f"https://instagram.com/{user}")

    time.sleep(3)

    # Take a screenshot of the website;
    driver.save_screenshot("screenshot.png")
    
    # lower the resolution of the image
    from PIL import Image
    img = Image.open("screenshot.png")
    img = img.resize((img.width // 2, img.height // 2))
    img.save("screenshot.png")


    # Close the browser
    driver.quit()

In [ ]:
from tqdm import tqdm 

for record in tqdm(records):
    # Access the fields of the record
    fields = record['fields']

    print(fields["name"])

    download_insta(fields["insta_handle"].replace("@", ""))
    
    # Use OpenAI to generate a summary
    summary = summarize_object_with_chatgpt(fields)

    descrip.create({

        "people": [record['id']],
        "desc": summary[0],
        "want": summary[1],
        "Name": fields["name"]

    })
